In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import auc

In [ ]:
df = list()

df_deeptfactor = pd.read_csv(snakemake.input['deepfactor_roc'])
df_deeptfactor['method'] = 'DeepTFactor'
df.append({
    'method': 'DeepTFactor',
    'auc': auc(df_deeptfactor['fpr'], df_deeptfactor['tpr']),
    'lora': 'DeepTFactor',
})

for i in snakemake.input['roc']:
    _df = pd.read_csv(i)
    df.append({
        'method': i.split('=')[1].split('_')[0].replace('e', ''),
        'auc': auc(_df.fpr, _df.tpr),
        'lora': 'Head Only'
    })

for i in snakemake.input['roc_lora']:
    _df = pd.read_csv(i)
    df.append({
        'method': i.split('=')[1].split('_')[0].replace('e', ''),
        'auc': auc(_df.fpr, _df.tpr),
        'lora': 'Head + LoRA'
    })

df = pd.DataFrame(df)
df['method'] = pd.Categorical(df['method'], categories=list(df['method'].unique()), ordered=True)
df['lora'] = pd.Categorical(df['lora'], categories=['DeepTFactor', 'Head Only', 'Head + LoRA'], ordered=True)

In [ ]:
df.rename(columns={'method': 'Model Size', 'auc': 'AUPRC', 'lora': 'Method'}) \
    .to_csv(snakemake.output['table_roc'], index=False)

In [ ]:
plt.figure(figsize=(6, 4), dpi=300)
plot = sns.barplot(data=df, x='method', y='auc', hue='lora', order=df['method'].cat.categories, hue_order=df['lora'].cat.categories)

for i, bar in enumerate(plot.patches):
    current_width = bar.get_width()
    current_pos = bar.get_x()
    
    if i == 0:
        bar.set_x(current_pos + current_width)
    else:
        bar.set_x(current_pos - current_width / 2)

plt.legend(loc='lower right', title='Method')
plt.xlabel('Method')
plt.ylabel('AUROC')
sns.despine()
plt.savefig(snakemake.output['roc'], bbox_inches='tight', dpi=300, transparent=True)

In [ ]:
df = list()

df_deeptfactor_pr = pd.read_csv(snakemake.input['deepfactor_pr'])
df_deeptfactor_pr['method'] = 'DeepTFactor'
df.append({
    'method': 'DeepTFactor',
    'auc': auc(df_deeptfactor_pr['recall'], df_deeptfactor_pr['precision']),
    'lora': 'DeepTFactor',
})

for i in snakemake.input['pr']:
    _df = pd.read_csv(i)
    df.append({
        'method': i.split('=')[1].split('_')[0].replace('e', ''),
        'auc': auc(_df['recall'], _df['precision']),
        'lora': 'Head Only'
    })

for i in snakemake.input['pr_lora']:
    _df = pd.read_csv(i)
    df.append({
        'method': i.split('=')[1].split('_')[0].replace('e', ''),
        'auc': auc(_df['recall'], _df['precision']),
        'lora': 'Head + LoRA'
    })

df = pd.DataFrame(df)
df['method'] = pd.Categorical(df['method'], categories=list(df['method'].unique()), ordered=True)
df['lora'] = pd.Categorical(df['lora'], categories=['DeepTFactor', 'Head Only', 'Head + LoRA'], ordered=True)

In [ ]:
df.rename(columns={'method': 'Model Size', 'auc': 'AUPRC', 'lora': 'Method'}) \
    .to_csv(snakemake.output['table_precision_recall'], index=False)

In [ ]:
plt.figure(figsize=(6, 4), dpi=300)
plot = sns.barplot(data=df, x='method', y='auc', hue='lora')
plt.legend(loc='lower right', title='Method')
plt.xlabel('Method')
plt.ylabel('AUPRC')
sns.despine()

for i, bar in enumerate(plot.patches):
    current_width = bar.get_width()
    current_pos = bar.get_x()
    
    if i == 0:
        bar.set_x(current_pos + current_width)
    else:
        bar.set_x(current_pos - current_width / 2)

plt.savefig(snakemake.output['precision_recall'], bbox_inches='tight', dpi=300, transparent=True)